## Code For Fully Frozen Bert Weights (Regression)
- The patients here have not been separated based on whether they died

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from multimodal import MultimodalDataset, LOSNetWeighted, collation
from sklearn.preprocessing import StandardScaler
from transformers import AutoModel
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [2]:
!nvidia-smi

Sat Apr  6 13:41:12 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   25C    P8    17W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
base_path = 'data/regression/no-outliers/combined'

static_train = pd.read_csv(f'{base_path}/static_train.csv')
static_val = pd.read_csv(f'{base_path}/static_val.csv')
static_test = pd.read_csv(f'{base_path}/static_test.csv')

In [4]:
static_train.shape

(2311, 25)

In [5]:
notes = pd.read_csv('data/notes_with_interval.csv')
notes_train = notes[notes['id'].isin(static_train['id'])]
notes_val = notes[notes['id'].isin(static_val['id'])]
notes_test = notes[notes['id'].isin(static_test['id'])]

In [6]:
dynamic = pd.read_csv('data/dynamic_cleaned.csv')
dynamic_train = dynamic[dynamic['id'].isin(static_train['id'])].copy()
dynamic_val = dynamic[dynamic['id'].isin(static_val['id'])].copy()
dynamic_test = dynamic[dynamic['id'].isin(static_test['id'])].copy()

In [7]:
features = ['aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium']

scaler = StandardScaler()

dynamic_train.loc[:, features] = scaler.fit_transform(dynamic_train[features])
dynamic_val.loc[:, features] = scaler.transform(dynamic_val[features])
dynamic_test.loc[:, features] = scaler.transform(dynamic_test[features])  

dynamic_train.head()

,id,charttime,aniongap,bicarbonate,bun,calcium,chloride,creatinine,glucose,sodium,potassium
0,26115624,9/7/50 0:22,-0.712744,-0.319670,-1.032452,-0.650622,1.438007,-0.734532,-0.565091,0.725645,-0.900854
1,21792938,4/13/28 14:18,-0.931146,0.794023,0.371756,-1.520537,-0.679054,1.532324,-0.588363,-1.249342,1.369643
2,28398464,12/9/34 8:10,0.597668,-0.505286,-0.463178,1.959125,-0.114505,-0.683012,-0.611635,-0.351621,-0.333230
6,21525925,2/10/45 5:40,-0.275940,-0.134054,-1.146306,-0.541882,0.591182,-0.837571,-0.378918,0.187012,-0.758948
7,22576776,1/13/84 7:30,1.252874,-0.505286,-1.146306,-0.106925,0.167770,-0.631493,-0.565091,0.546101,-0.617042


### Dynamic train cleaning

In [8]:
id_lengths_train = dynamic_train['id'].value_counts().to_dict()
dynamic_train = dynamic_train.sort_values(by=['id', 'charttime'])
dynamic_train = dynamic_train.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_train['id']).agg(list)

dynamic_train

id
20008098    [[0.5976678649580875, -0.8765166023203315, -0....
20013244    [[-0.49434217878227354, 0.6084077288686798, -0...
20015730    [[-0.931146196278418, 0.237176646071427, -0.72...
20020562    [[-0.931146196278418, 0.051561104672800596, -0...
20022095    [[4.528904022423387, -3.2895186405024748, -0.7...
                                  ...                        
29988601    [[0.8160698737061598, -0.3196699781244522, 0.5...
29990184    [[-0.0575381612861291, 1.9077165186590646, 0.2...
29990494    [[0.8160698737061598, -0.690901060921705, -0.4...
29994296    [[-0.49434217878227354, -0.8765166023203315, 1...
29997500    [[-0.7127441875303457, 2.65017868425357, -0.53...
Length: 2311, dtype: object

### Dynamic val cleaning

In [9]:
id_lengths_val = dynamic_val['id'].value_counts().to_dict()
dynamic_val = dynamic_val.sort_values(by=['id', 'charttime'])
dynamic_val = dynamic_val.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_val['id']).agg(list)

dynamic_val

id
20042619    [[-0.27594017003420135, 0.4227921874700534, -0...
20098037    [[-0.49434217878227354, 0.051561104672800596, ...
20111879    [[-0.931146196278418, 0.237176646071427, 0.144...
20197360    [[-0.0575381612861291, -0.13405443672582582, -...
20226142    [[-0.49434217878227354, 1.165254353064559, -0....
                                  ...                        
29669406    [[0.16086384746194313, 0.237176646071427, -0.9...
29742461    [[0.5976678649580875, -1.4333632265162106, 0.4...
29801793    [[-0.7127441875303457, -0.8765166023203315, 0....
29841097    [[-0.931146196278418, 0.7940232702673062, -0.7...
29862044    [[-1.804754231270707, 0.9796388116659326, -0.6...
Length: 257, dtype: object

### Dynamic test cleaning

In [10]:
id_lengths_test = dynamic_test['id'].value_counts().to_dict()
dynamic_test = dynamic_test.sort_values(by=['id', 'charttime'])
dynamic_test = dynamic_test.apply(lambda x: list(x[features]), axis=1).groupby(dynamic_test['id']).agg(list)

dynamic_test

id
20021110    [[0.16086384746194313, 0.6084077288686798, -0....
20030660    [[-0.931146196278418, -0.13405443672582582, -1...
20114766    [[-0.0575381612861291, -0.690901060921705, 0.4...
20128881    [[-0.0575381612861291, -0.3196699781244522, -0...
20134597    [[-0.0575381612861291, -0.5052855195230787, -0...
                                  ...                        
29865517    [[2.5632859436907376, -2.1758253921107165, 0.2...
29891413    [[-1.804754231270707, -0.13405443672582582, -0...
29907670    [[-0.7127441875303457, 0.051561104672800596, -...
29923363    [[-0.0575381612861291, -0.3196699781244522, -1...
29970938    [[-0.27594017003420135, -0.13405443672582582, ...
Length: 286, dtype: object

In [11]:
notes = notes[['id', 'charttime', 'text', 'interval']]

notes_train = notes[notes['id'].isin(static_train['id'])].copy()
notes_val = notes[notes['id'].isin(static_val['id'])].copy()
notes_test = notes[notes['id'].isin(static_test['id'])].copy()

In [12]:
train_data = MultimodalDataset(static=static_train, dynamic=dynamic_train, id_lengths=id_lengths_train, notes=notes_train)
validation_data = MultimodalDataset(static=static_val, dynamic=dynamic_val, id_lengths=id_lengths_val, notes=notes_val)

train_loader = DataLoader(train_data, batch_size=1500, shuffle=True, collate_fn=collation)
val_loader = DataLoader(validation_data, batch_size=150, shuffle=True, collate_fn=collation)

In [13]:
text_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

for params in text_model.parameters():
    params.requires_grad = False

model = LOSNetWeighted(input_size=9, out_features=1, hidden_size=128, text_model=text_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f'device: {device}')

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


device: cuda


In [14]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
    
    def forward(self, predicted, actual):
        return torch.sqrt(self.mse(predicted, actual))

In [15]:
criterion = RMSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
!nvidia-smi

Sat Apr  6 13:41:31 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   28C    P2    62W / 300W |   2519MiB / 49140MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
loss_base_path = 'losses/no-outliers/baseline_regression'

In [18]:
epochs = 200
training_loss = []
validation_loss = []
patience = 10
stagnation = 0

for epoch in range(1, epochs+1):
    print(f'training epoch: [{epoch}/{epochs}]')
    model.train()
    training_loss_epoch = 0

    for step, batch in enumerate(train_loader):
        packed_dynamic_X, notes_X, notes_intervals, los = batch

        packed_dynamic_X = packed_dynamic_X.to(device)
        los = los.to(device)

        notes_X_gpu = []
        for notes in notes_X:
            notes_gpu = {key: value.to(device) for key, value in notes.items()}
            notes_X_gpu.append(notes_gpu)

        outputs = model(packed_dynamic_X, notes_X_gpu, notes_intervals)

        loss = criterion(outputs, los)
        training_loss_epoch += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % max(1, round(len(train_loader) * 0.1)) == 0:
            print(f'step: [{step+1}/{len(train_loader)}] | loss: {loss.item():.4}')

            if step+1 == 1 and epoch == 1:
                with open(f'{loss_base_path}/loss_step.txt', 'w') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

            else:
                with open(f'{loss_base_path}/loss_step.txt', 'a') as loss_step_f:
                    loss_step_f.write(f'{loss.item():.4f}\n')

    avg_training_loss_epoch = training_loss_epoch / len(train_loader)
    training_loss.append(avg_training_loss_epoch.item())
    print(f'Training epoch loss: {avg_training_loss_epoch.item():.4f}\n')

    if epoch == 1:
        with open(f'{loss_base_path}/training_loss_epoch.txt', 'w') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    else:
        with open(f'{loss_base_path}/training_loss_epoch.txt', 'a') as loss_epoch_train_f:
            loss_epoch_train_f.write(f'{avg_training_loss_epoch.item():.4f}\n')

    print(f'validation epoch: [{epoch}/{epochs}]')
    
    model.eval()
    with torch.no_grad():
        validation_loss_epoch = 0

        for val_step, val_batch in enumerate(val_loader):
            packed_dynamic_X_val, notes_X_val, notes_intervals_val, los_val = val_batch

            packed_dynamic_X_val = packed_dynamic_X_val.to(device)
            los_val = los_val.to(device)

            notes_X_val_gpu = []
            for notes in notes_X_val:
                notes_val_gpu = {key: value.to(device) for key, value in notes.items()}
                notes_X_val_gpu.append(notes_val_gpu)

            val_outputs = model(packed_dynamic_X_val, notes_X_val_gpu, notes_intervals_val)
            val_loss = criterion(val_outputs, los_val)
            validation_loss_epoch += val_loss

        avg_validation_loss = validation_loss_epoch / len(val_loader)
        print(f'Validation epoch loss: {avg_validation_loss.item():.4f}\n')
        
        if len(validation_loss) == 0 or (avg_validation_loss.item() < min(validation_loss)):
            stagnation = 0
            torch.save(model.state_dict(), 'saved-models/no-outliers/baseline_regression_best_model.pth')
            print(f'new minimum validation loss')
            print(f'model saved\n')

        else:
            stagnation += 1

        validation_loss.append(avg_validation_loss.item())

        if epoch == 1:
            with open(f'{loss_base_path}/validation_loss_epoch.txt', 'w') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_validation_loss.item():.4f}\n')

        else:
            with open(f'{loss_base_path}/validation_loss_epoch.txt', 'a') as loss_epoch_val_f:
                loss_epoch_val_f.write(f'{avg_validation_loss.item():.4f}\n')

        if stagnation >= patience:
            print(f'No improvement over {patience} epochs')
            print('Early stopping')
            break

    model.train()

    print('===============================\n')
    

training epoch: [1/200]
step: [1/2] | loss: 4.543
step: [2/2] | loss: 4.364
Training epoch loss: 4.4536

validation epoch: [1/200]
Validation epoch loss: 3.9128

new minimum validation loss
model saved


training epoch: [2/200]
step: [1/2] | loss: 3.908
step: [2/2] | loss: 3.436
Training epoch loss: 3.6722

validation epoch: [2/200]
Validation epoch loss: 3.1915

new minimum validation loss
model saved


training epoch: [3/200]
step: [1/2] | loss: 3.09
step: [2/2] | loss: 2.943
Training epoch loss: 3.0162

validation epoch: [3/200]
Validation epoch loss: 2.7383

new minimum validation loss
model saved


training epoch: [4/200]
step: [1/2] | loss: 2.703
step: [2/2] | loss: 2.631
Training epoch loss: 2.6668

validation epoch: [4/200]
Validation epoch loss: 2.7653


training epoch: [5/200]
step: [1/2] | loss: 2.679
step: [2/2] | loss: 2.796
Training epoch loss: 2.7373

validation epoch: [5/200]
Validation epoch loss: 2.9586


training epoch: [6/200]
step: [1/2] | loss: 2.863
step: [2/2] |